# Build a Dashboard Application

In this lab, we will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. We will be guided to build this dashboard application via the following tasks:

* **TASK 1:** Add a Launch Site Drop-down Input Component
* **TASK 2:** Add a callback function to render `success-pie-chart` based on selected site dropdown
* **TASK 3:** Add a Range Slider to Select Payload
* **TASK 4:** Add a callback function to render the `success-payload-scatter-chart` scatter plot

## Download a skeleton dashboard application and dataset

First, let's get the SpaceX Launch dataset for this lab

Let's first import required Python packages for this lab:

In [1]:
!pip3 install pandas dash

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import wget

<ipython-input-2-7f16c1f1e63e>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-7f16c1f1e63e>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()

100% [................................................................................] 2476 / 2476

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


Read the airline data into pandas dataframe

In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

Create a dash application

In [5]:
app = dash.Dash(__name__)

Create an app layout

In [6]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

In [7]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(data, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
        names='class',
        title=f'Total Success Launched for site {entered_site}')
        return fig

Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

In [8]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload):
    low, high = payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]       
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
        x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_5_1m1656868587.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/deps/react@16.v2_5_1m1656868587.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_5_1m1656868587.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_5_1m1656868587.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_5_1m1656868587.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_5_0m1656868587.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_dash-component-suites/d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kgcgo\anaconda3\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_d

127.0.0.1 - - [26/Nov/2022 15:28:50] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Nov/2022 15:28:50] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:28:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:29:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:29:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2022 15:29:09] "POST /_dash-update-component HTTP/1.1" 200 -
